In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
df = pd.read_csv('/kaggle/input/google-stock-price-all-time/google.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
fig = px.line(df, x='Date', y="Close",title='Google Close Prices All Time')


fig.add_vline(x ='2020-04-01', line_dash='dot')

fig.update_xaxes(rangeslider_visible=True)
fig.update_traces(line=dict(color='gold'))

fig.update_layout(margin={'b':0,'l':0,'r':0,},
                  paper_bgcolor='rgb(248, 248, 255)',
                  plot_bgcolor='rgb(248, 248, 255)',
                  title={'font':{
                             'family':'monospace',
                             'size':22,
                             'color': 'grey'
                         },
                        'x':0.45,'y':0.9})

fig.show()

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                                    open=df['Open'], 
                                    high=df['High'],
                                    low=df['Low'],
                                    close=df['Close'],
                                    increasing_line_color = 'black',
                                    decreasing_line_color = 'orange')
                                    ])

fig.update_layout(margin={'b':0,'l':0,'r':0,},
                  paper_bgcolor='rgb(248, 248, 255)',
                  plot_bgcolor='rgb(248, 248, 255)',
                  title={'font':{
                             'family':'monospace',
                             'size':22,
                             'color': 'grey'
                         },
                         'text': 'Google Prices All Time',
                        'x':0.45,'y':0.9})
fig.show()

In [ ]:
df1 = df['Close']
df1 = scaler.fit_transform(np.array(df1).reshape(-1,1))

training_size = int(len(df1)*0.70)
test_size = len(df1) - training_size

train_data,test_data = df1[0:training_size, :], df1[training_size:len(df1),:1]

In [ ]:
import numpy
def create_dataset(dataset,time_step=1):
    x,y = [],[]
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step),0]
        x.append(a)
        y.append(dataset[i + time_step,0])
    return numpy.array(x), numpy.array(y)

In [ ]:
time_step = 100
x_train, y_train = create_dataset(train_data, time_step)
x_test, y_test = create_dataset(test_data, time_step)

x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(x_train,y_train, validation_data=(x_test,y_test),batch_size=64,verbose=1,epochs=100)

In [ ]:
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)



In [ ]:
train_pred = scaler.inverse_transform(train_pred)
test_pred = scaler.inverse_transform(test_pred)


In [ ]:
math.sqrt(mean_squared_error(y_train, train_pred))

In [ ]:
math.sqrt(mean_squared_error(y_test, test_pred))